In [ ]:
import nltk
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.svm import LinearSVC
from gensim.models import KeyedVectors
from tqdm import tqdm

In [ ]:
sentiment_data = pd.read_csv("./train.csv")

In [ ]:
DATA_TO_USE = 250000

texts = sentiment_data['message'].tolist()[:DATA_TO_USE]
labels = np.array(sentiment_data['sentiment'])[:DATA_TO_USE]

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, random_state=21)

## Exercise 3: word vectors meet bag of words

In this exercise you use your newly trained word vectors and a simple Bag of Words models to approach the sentiment analysis task

In [ ]:
# We will use a convinient wrapper for our word2vec model provided by gensim

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format("./simple_cbow.w2v")

In [ ]:
# you can get the vector for a word in a simple way
w2v_model['word']

In [ ]:
# you can easily query the model for word most similar to a give word 
w2v_model.most_similar('funny')

### 3.1

In this exercise you learn how to encode sentences with word2vec using a bag of words approach

In [ ]:
# implement a tokenizer that you will use throughout the exercise
# I would recommend a regexp tokenizer for speed, but it's completely up to you
def my_tokenizer(text):
    return nltk.regexp_tokenize(text, '\w+')

In [ ]:
def bow_encoder(wmodel, tokenizer, text):
    """
    This function encodes text into a vector.
    
    First, it tokenizes input text using the provided tokenizer function.
    Then it uses the provided word2vec model to get the vectors corresponding to text's tokens.
    Finally, it computes an average of all token's vectors and returns it.
    
    If the function failed to find and encode any words, it should at least return a vector of zeros.
    """
    zero_vector = np.zeros(w2v_model.vector_size)
    # your code goes here
    return sent_vector

Now use your new encoder to encode both train_texts and test_texts into matrices.

The number of rows in a matrix should be equal to the number of texts encoded.

The number of columns should be equal to the word2vec space dimansionality (currently = 128)

Just write a little loop.

In [ ]:
train_encoded = 
test_encoded = 

In [ ]:
assert isinstance(train_encoded, np.ndarray)
assert isinstance(test_encoded, np.ndarray)

assert train_encoded.shape[0] == len(train_texts)
assert train_encoded.shape[1] == w2v_model.vector_size

assert test_encoded.shape[0] == len(test_texts)
assert test_encoded.shape[1] == w2v_model.vector_size
print('done')

In [ ]:
clf = LinearSVC()
clf.fit(train_encoded, train_labels)
preds = clf.predict(test_encoded)

print(classification_report(test_labels, preds))
print("AUC = {}".format(roc_auc_score(test_labels, preds)))

### Not too impressive!

### 3.2

In this exercise you attempt to improve your encoder by filtering out stop words.

In [ ]:
def bow_encoder_with_stopwords(wmodel, tokenizer, stopwords, text):
    """
    This function encodes text into a vector.
    
    First, it tokenizes input text using the provided tokenizer function.
    Then it removes any stopwords from the list of tokens.
    Then it uses the provided word2vec model to get the vectors corresponding to text's tokens.
    Finally, it computes an average of all token's vectors and returns it.
    
    If the function failed to find and encode any words, it should at least return a vector of zeros.
    """
    zero_vector = np.zeros(w2v_model.vector_size)
    # your code goes here
    return sent_vector

In [ ]:
stops = set(nltk.corpus.stopwords.words('english'))

In [ ]:
train_encoded = 
test_encoded = 

In [ ]:
clf = LinearSVC()
clf.fit(train_encoded, train_labels)
preds = clf.predict(test_encoded)

print(classification_report(test_labels, preds))
print("AUC = {}".format(roc_auc_score(test_labels, preds)))

## Looks like the BoW model is not too good for the job!

![architecture](pics/we_need_to_go_deeper.jpg)

## Introducing: Keras

Keras is a cool library built on top of the computational backend provided by Tensorflow. It provides a layer of abstraction between you and complicated tensor algebra, allowing for rapid prototyping of deep neural networks.

### 3.3: Data preparation

Before we start crunching word vectors with convolutional neural networks, we need to prepare our data.

In [ ]:
import keras

In [ ]:
# load the vocabulary we created earlier
voc, rvoc = pickle.load(open("./dict_rdict.pkl","rb"))

In [ ]:
# we are going to use the whole dataset this time around
texts = sentiment_data['message'].tolist()
labels = np.array(sentiment_data['sentiment'])

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, random_state=21)

Recall the function that turns tokens into their ids

In [ ]:
# hint: you may want to use the function you've built during seminar 2
MAX_LEN = 32

def vectorize_tokens(sentence, tokenizer, token_to_id, max_len):
    """
    Preprocesses a sentence into list of tokens using the provided tokenizer
    Then converts it into a list of token ids using the supplied 'token_to_id' dictionary.
    Pads resulting list with NULL identifiers up to max_len length. 
    """
    # your code goes here
    # STEP 1: convert sentence to a list of tokens
    
    # STEP 2: replace tokens with their identifiers from the vocabulary
    # If the token is not present in the vocabulary, replace it with UNKN identifier

    # STEP 3: pad the sequence id's with NULL identifiers until so that it's length is equal to max_len

    return ids

Now apply the vectorization function to every sentence from train and test datasets. In the end you should end up with a matrix of shape [len(data), MAX_LEN].

Just write a little loop

In [ ]:
def vectorize_sentences(sentences, tokenizer, token_to_id, max_len):
    sentence_ids = []
    
    # your code goes here
        
    return np.array(sentence_ids)

In [ ]:
train_vectorized = vectorize_sentences(train_texts, my_tokenizer, voc, MAX_LEN)
test_vectorized = vectorize_sentences(test_texts, my_tokenizer, voc, MAX_LEN)

In [ ]:
assert isinstance(train_vectorized, np.ndarray)
assert isinstance(test_vectorized, np.ndarray)

assert train_vectorized.shape == (len(train_vectorized), MAX_LEN)
assert test_vectorized.shape == (len(test_vectorized), MAX_LEN)

print('done')

### 3.4 Building a deep NN

In [ ]:
embeddings_matrix = w2v_model.syn0

In [ ]:
# keras Input layer is basically the same thing as tf.placeholder
# it defines a node where the network will be expecting to recieve input data
input_layer = keras.layers.Input(shape=(MAX_LEN,))

In [ ]:
# keras Embedding layer is a container for dense vectors
# it recieves a list of token identifiers of shape [MAX_LEN] 
# and turns it into a matrix of shape [MAX_LEN, EMBEDDING_DIM]

embedding_layer = keras.layers.Embedding(embeddings_matrix.shape[0], embeddings_matrix.shape[1], 
                                         input_length=MAX_LEN, weights=[embeddings_matrix],
                                         trainable=False)(input_layer)
# notice how the input_layer is plugged into the embedding_layer

In [ ]:
# keras Convolutional layer implements a set of learnable filters
# that extract local patterns from input data
convolution_layer = keras.layers.Convolution1D(128, 3)(embedding_layer)

In [ ]:
# keras GlobalMaxPooling layer applies a max filter to the input feature representation
# only the strongest responses from the previous layer are kept, everything else is discarded
subsampling_layer = keras.layers.GlobalMaxPooling1D()(convolution_layer)

In [ ]:
# keras Linear layers apply a simple linear transformation to input data, 
# which is optionally followed by a non-linear activation function
# very useful for building Multi-Layer Perceptrons
linear_layer_1 = keras.layers.Dense(64, activation='relu')(subsampling_layer)
linear_layer_2 = keras.layers.Dense(1, activation='sigmoid')(linear_layer_1)

In [ ]:
# this compiles the computational graph we've just created, applies a loss function
# and pre-computes the gradients for back propagation

deep_model = keras.models.Model(inputs=[input_layer], outputs=[linear_layer_2])
deep_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
deep_model.summary()

In [ ]:
deep_model.fit(x=train_vectorized, y=train_labels, batch_size=64, epochs=3, 
               validation_data=[test_vectorized, test_labels])

In [ ]:
preds = deep_model.predict(test_vectorized)

In [ ]:
print("AUC = {}".format(roc_auc_score(test_labels, preds)))

In [ ]:
deep_model.save_weights("nn_weights.hdf5")

### Thats more like it! Keep in mind that we only trained a tiny (57k parameters) model because of the limitations of CPU computing power. Using a deeper model with more trainable filters in the Convolution layer would likely result in even stronger predictive power. Stay tuned! 